In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
db = client.performance_dashboard

In [2]:
%matplotlib inline
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
import ipywidgets as widgets
from IPython.display import display

components = db.benchmark_results.distinct("component")
component_selection = widgets.Dropdown(description="Component", options=components, value='Ocean')
display(component_selection)

In [8]:
from IPython.display import Javascript, display
# from ipywidgets import widgets

def run_all(ev):
    global component
    component = component_selection.value   
    display(Javascript('IPython.notebook.execute_cells_below()'))

component = component_selection.value
button = widgets.Button(description="Update below")
button.on_click(run_all)
display(button)

In [9]:
from bokeh.charts import Scatter
from bokeh.plotting import show
from bokeh.models.formatters import NumeralTickFormatter

def bokeh_scatter(component, benchmark_name, benchstep_name, df):
    title = "{} {} - {}".format(component, benchmark_name, benchstep_name)
    p = Scatter(df, x='revision', y='value', color='version', marker='version',
                title=title, legend="top_left",
                xlabel="Revision", ylabel="Time")
    p.xaxis[0].formatter = NumeralTickFormatter(format="0")
    return p

In [10]:
from bokeh.io import show
from IPython.display import clear_output

def bench_generator():
    benchmark_names = db.benchmark_results.find({"component": component}).distinct("benchmark_name")
    for benchmark_name in benchmark_names:
        benchstep_names = db.benchmark_results.find({"component": component,
                                                     "benchmark_name": benchmark_name
                                                    }
                                                   ).distinct("benchstep_name")
        for benchstep_name in benchstep_names:
            versions = db.benchmark_results.find({"component": component,
                                                  "benchmark_name": benchmark_name,
                                                  "benchstep_name": benchstep_name,
                                                 }
                                                ).distinct("version")
            versions.sort(reverse=True)


            query = {"component": component, "benchmark_name": benchmark_name, 
                     "benchstep_name": benchstep_name, "version": {"$in": versions}}
            df = pd.DataFrame.from_records(db.benchmark_results.find(query))
            # df.info()
            # bokeh_scatter(component, benchmark_name, benchstep_name, df)
            p = bokeh_scatter(component, benchmark_name, benchstep_name, df)
            clear_output()
            show(p)
            yield


In [11]:
import ipywidgets as widgets

benchs = bench_generator()

previous_bench = widgets.Button(
    description='<',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Previous',
    icon='check'
)
play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=200000,
    step=1,
    description="Press play",
    disabled=False
)
next_bench = widgets.Button(
    description='>',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Next',
    icon='check'
)

def on_button_clicked(b):
    global benchs
    next(benchs)

next_bench.on_click(on_button_clicked)


In [13]:
widgets.HBox([previous_bench, play, next_bench])